In [1]:
import glob
import os

import codecs
import pandas as pd

In [2]:
os.chdir("./hospital-report")

In [82]:
pd.set_option('display.max_columns', 40)

In [53]:
# score = pd.read_csv('調査対象病院.csv', encoding="utf-8_sig")
score = pd.read_csv('scores/調査対象病院.csv', encoding="shift_jis")

In [65]:
score["区"]=score['住所'].str.split('区', expand=True)[0]+"区"
score.loc[score["総病床数"]>=500, '規模']='大病院'
score.loc[score["総病床数"]<=499, '規模']='中病院'
score.loc[score["総病床数"]<100, '規模']='小病院'
score.to_csv('調査対象病院.csv', encoding="shift_jis", index=False)

## 4つの原則に基づいたスコアの集計

- 全体の集計
- 区毎の集計
- 病床数毎の集計

In [91]:
score.describe().to_csv('scores/全病院.csv', encoding='shift_jis')

In [105]:
wards = ['中央区', '千代田区', '台東区', '文京区', '港区']
for w in wards:
    name = f"scores/{w}.csv"
    score[score['区']==w].describe().to_csv(name, encoding='shift_jis')

In [104]:
scales = ['小病院', '中病院', '大病院']
for s in scales:
    name = f"scores/{s}.csv"
    score[score['規模']==s].describe().to_csv(name, encoding='shift_jis')

In [61]:
# files
files = glob.glob('./unachieved-items/*')
for name in files:
    report = pd.read_csv(name, encoding="shift_jis")

In [68]:
report[report['種別']=="問題あり"]['JIS']

0               A: 1.1.1
1               A: 1.1.1
2               A: 1.1.1
3               A: 1.1.1
4               A: 1.1.1
5               A: 1.1.1
6               A: 1.1.1
7               A: 1.1.1
8               A: 1.1.1
9               A: 1.1.1
10              A: 1.1.1
11              A: 1.1.1
12              A: 1.1.1
13    A: 1.1.1, A: 2.4.4
17              A: 3.1.1
18    A: 2.4.1, A: 4.1.2
33              A: 4.1.1
Name: JIS, dtype: object